# Future Competitions

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/wca-ipy/

## Initialisation

Basic approach to determine the project directory

In [36]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

try:
    debug = int(os.environ['LOGIQX_DEBUG'])
except:
    debug = 0

## Generic Class

Generic class to ensure that all custom classes are printable

In [37]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## CSV Reader

Generic function to read CSVs into memory as actual objects

In [38]:
import csv

def readCsv(extractName, classType):
    objects = []

    fn = os.path.join(projdir, 'data', 'extract', '%s.csv' % extractName)
    with open(fn, 'r') as f:
        csvReader = csv.reader(f)

        for inputRow in csvReader:
            objects.append(classType(inputRow))

        return objects

## Load Persons

Load seniors into memory

In [39]:
class Country(Printable):
    def __init__(self, fields):
        self.id, self.name, self.continent = fields

countries = {}
for country in readCsv('countries', Country):
    countries[country.id] = country

In [40]:
class Person(Printable):
    def __init__(self, fields):
        self.id, self.name, self.country, self.username, self.usernum, self.age, self.hidden, self.userId = fields
        self.usernum = int(self.usernum)

    def getWcaLink(self, eventId = None):
        if (eventId):
            link = '<a href="https://www.worldcubeassociation.org/persons/%s#%s">%s</a>' % (self.id, eventId, self.name)
        else:
            link = '<a href="https://www.worldcubeassociation.org/persons/%s">%s</a>' % (self.id, self.name)

        return link

    def getSpeedsolvingLink(self):
        if self.usernum > 0:
            profile = '%s.%d' % (self.username.replace(' ', '-').replace('.', '-').lower(), self.usernum)
            link = '<a href="https://www.speedsolving.com/members/%s">%s</a>' % (profile, self.username)
        else:
            link = ''

        return link

persons = {}
users = {}
for person in readCsv('seniors', Person):
    if person.hidden == 'y':
        if debug:
            person.name = '* %s *' % person.name
            persons[person.id] = person
            users[int(person.userId)] = person
    else:
        persons[person.id] = person
        users[int(person.userId)] = person

## Process Competitions

Read competitions into memory, prior to processing

In [41]:
import datetime
import urllib.request
import json

from bs4 import BeautifulSoup

class Competition(Printable):
    def __init__(self, fields):
        """Initialisise the partial results"""
        self.compId, self.compName, self.compCity, self.compCountry, self.compWebsite, self.startDate, self.endDate = fields


    def getAttendees(self):
        """Download the list of attendees"""

        def parseHtml(st):
            soup = BeautifulSoup(st, "lxml")
            anchors = soup.find_all("a")

            for anchor in anchors:
                href = anchor.get("href")
                if href:
                    personId = ''
                    if 'worldcubeassociation.org/persons/' in href:
                        personId = href[-10:]
                    elif 'worldcubeassociation.org/results/p.php?i=' in href:
                        personId = href[-10:]
                    elif 'cubingchina.com/results/person/' in href:
                        personId = href[-10:]

                    if personId in persons.keys():
                        self.attendees.append(persons[personId])

        def parseJson(st):           
            jsonTxt = response.read().decode('utf-8')
            registrations = json.loads(jsonTxt)

            for registration in registrations:
                userId = registration['user_id']
                if userId in users.keys():
                    self.attendees.append(users[userId])

        self.attendees = []

        comp_url = ''

        # This list is deprecated as they are now using the WCA website for registrations
        # speedcubing.nz/competitors
        # canadiancubing.com/Competitors
        
        # zawody4event.pl is still being used for Poland 2020-01-07 but I am unable to connect from my EC2 instance
        # if 'zawody4event.pl' in self.compWebsite:
            # comp_url = self.compWebsite

        # speedcubing.pl is still being used for Poland 2020-01-07
        if 'speedcubing.pl' in self.compWebsite:
            comp_url = self.compWebsite
        # cubingchina.com is still being used for China 2020-01-07
        elif 'cubingchina.com' in self.compWebsite:
            comp_url = self.compWebsite + '/competitors'
        # cubing-tw.net is still being used for Taiwan 2020-01-07
        elif 'cubing-tw.net' in self.compWebsite:
            comp_url = self.compWebsite + '/competitors'
        # cubecomp.de is possibly being used by Switzerland 2020-01-07 (Fast Fingers 2020)
        elif 'cubecomp.de' in self.compWebsite:
            comp_url = self.compWebsite + '/competitors'

        numAttempts = 1
        done = False
        
        while not done:
            try:
                if comp_url:
                    req = urllib.request.Request(comp_url, headers={'User-Agent': 'Mozilla'})
                    response = urllib.request.urlopen(req, timeout = 15)
                    parseHtml(response)

                else:
                    comp_url = "https://www.worldcubeassociation.org/api/v0/competitions/%s/registrations" % self.compId

                    req = urllib.request.Request(comp_url, headers={'User-Agent': 'Mozilla'})
                    response = urllib.request.urlopen(req, timeout = 15)
                    parseJson(response)

                done = True

            except urllib.error.HTTPError as e:
                if ('www.worldcubeassociation.org' in comp_url or
                       'cubingchina.com' in comp_url) and e.code == 404:
                    print("Skipping %s - attempt %d (%d)" % (comp_url, numAttempts, e.code))
                    return 1

                print("Failed to retrieve %s - attempt %d (%d)" % (comp_url, numAttempts, e.code))

                if numAttempts < 3:
                    numAttempts += 1
                else:
                    raise e

            except:
                print("Failed to retrieve %s - attempt %d" % (comp_url, numAttempts))

                if numAttempts < 3:
                    numAttempts += 1
                else:
                    raise
    
        return 0
                    

    def listAttendees(self, weekNo, indent = ' ' * 6):
        """List seniors from memory"""

        if len(self.attendees) > 0:
            link = '<a href="https://www.worldcubeassociation.org/competitions/%s">%s</a>' % (self.compId, self.compName)
            startDate = datetime.datetime.strptime(self.startDate, '%Y-%m-%d')
            endDate = datetime.datetime.strptime(self.endDate, '%Y-%m-%d')
            if startDate.day == endDate.day:
                dates = startDate.strftime('%b %-d, %Y')
            elif startDate.month == endDate.month:
                dates = startDate.strftime('%b %-d') + endDate.strftime(' - %-d, %Y')
            elif startDate.year == endDate.year:
                dates = startDate.strftime('%b %-d') + endDate.strftime(' - %b %-d, %Y')
            else:
                dates = startDate.strftime('%b %-d, %Y') + endDate.strftime(' - %b %-d, %Y')

            if weekNo == 1:
                html = '%s<details open>\n' % indent
            else:
                html = '%s<details>\n' % indent
            html += '%s  <summary><i class="flag flag-%s"></i>&nbsp;%s - %s, %s - %s</summary>\n' % \
                (indent, self.compCountry, link, self.compCity, countries[self.compCountry].name, dates)
            html += '%s  <p style="margin-left: 20px">\n' % indent

            count = 0
            for person in sorted(self.attendees, key=lambda person: person.name):
                if count:
                    html += '%s<br/>\n' % indent

                html += '    %s<i class="flag flag-%s"></i>&nbsp; %s, %s' % \
                    (indent + '&nbsp;' * 4, person.country, person.getWcaLink(), countries[person.country].name)
                if person.usernum > 0:
                    html += ' - %s on Speedsolving.com' % person.getSpeedsolvingLink()
                
                count += 1

            html += '\n'
            html += '%s  </p>\n' % indent
            html += '%s</details>\n\n' % indent

        else:
            html = ''
            
        return html
    
competitions = []
for competition in readCsv('competitions', Competition):
    if (datetime.datetime.strptime(competition.endDate, '%Y-%m-%d') >= datetime.datetime.now()):
        competitions.append(competition)

competitions = sorted(competitions, key=lambda c: (c.startDate, c.endDate, c.compName))

## Analyse Competitions

Process the competitions one-by-one

In [42]:
import time
pc1 = time.perf_counter()

fn = 'Future_Competitions.html'
with open(os.path.join(projdir, 'templates', fn), 'r') as f:
    html = ''.join(f.readlines())

refreshed = datetime.datetime.now().replace(microsecond=0).isoformat().replace('T', ' ')
html = html.replace('REPLACE_WITH_TIMESTAMP', refreshed)

fn = 'known_senior_details'

content = ''
prevDate = ''
weekNo = 0
skips = 0

for competition in competitions:
    skips += competition.getAttendees()
    if skips > 3:
        raise Exception('Too many skips!')

    if len(competition.attendees) > 0:
        endDate = datetime.datetime.strptime(competition.endDate, '%Y-%m-%d')
        endDate += datetime.timedelta(days = 6 - (endDate.weekday() + 6) % 7)
        if endDate != prevDate:
            startDate = endDate - datetime.timedelta(days = 7)
            if startDate.year == endDate.year:
                content += '<h3>%s - %s</h3>\n\n' % (startDate.strftime('%b %-d'), endDate.strftime('%b %-d, %Y'))
            else:
                content += '<h3>%s - %s</h3>\n\n' % (startDate.strftime('%b %-d, %Y'), endDate.strftime('%b %-d, %Y'))
            prevDate = endDate
            weekNo += 1

        content += competition.listAttendees(weekNo)
    
html = html.replace('REPLACE_WITH_CONTENT', content)

fn = os.path.join(projdir, 'docs', 'Future_Competitions.html')
if not os.path.exists(os.path.dirname(fn)):
    os.makedirs(os.path.dirname(fn))
with open(fn, 'w') as f:
    f.write(html)

pc2 = time.perf_counter()
print("Competitions completed in %0.2f seconds" % (pc2 - pc1))

Competitions completed in 0.87 seconds


## All Done!